In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np
import torch
import matplotlib.pyplot as plt

from Inference import Inferencer

import IPython.display as ipd

In [2]:
inferencer = Inferencer(
    hp_path= 'Hyper_Parameters.yaml',
    checkpoint_path= './results/Checkpoint/S_10000.pt',
    batch_size= 4
    )

In [10]:
feature = np.load('./Inference_Test/test.npy')
feature = torch.from_numpy(pattern).T.unsqueeze(0)
feature_length = [feature.size(2)]

In [14]:
audio = inferencer.Inference(
    features= feature,
    feature_lengths= feature_length
    )[0]

In [15]:
ipd.display(ipd.Audio(audio, rate= inferencer.hp.Sound.Sample_Rate))